# Sample code of Image Classification VGG16 Model with PyTorch

This notebook is the sample code of training the image classification model using COCO2014 dataset.  
COCO2014 dataset has not classification labels, therefore it makes classification dataset cropping bounding boxes.

|Item|Description|
|---|---|
|DeepLearning Framework|PyTorch|
|Dataset|COCO2014 Classification|
|Model Architecture|VGG16|

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
#import cv2
#import json
#import numpy as np
#import pandas as pd
#from pathlib import Path
#from tqdm import tqdm
#from PIL import Image
#from data_loader.data_loader import DataLoader
#from models.pytorch import vgg16
#
#import torch
#from torch.utils.data import Dataset

import random
import numpy as np
import torch
import pprint

from data_loader.data_loader import DataLoader
from models.pytorch import vgg16

## Set Random Seed

In [3]:
seed=42

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

## Device Settings

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda', index=0)

## Hyperparameters

In [5]:
epochs = 50
batch_size = 32
learning_rate = 0.001
weight_decay = 0.001
input_tensor_shape = (3, 224, 224)   # CHW

## Preparing Dataset

### Download and Extract

In [6]:
dataset_dir = '/tmp/dataset'
os.makedirs(dataset_dir, exist_ok=True)
dataloader = DataLoader(dataset_name='coco2014_classification_pytorch', resize=input_tensor_shape[1:], dataset_dir=dataset_dir, batch_size=batch_size)

100% 1000/1000 [00:21<00:00, 46.78it/s]
100% 519/519 [00:00<00:00, 1303.07it/s]
100% 519/519 [00:00<00:00, 420159.00it/s]
100% 1000/1000 [00:16<00:00, 61.86it/s]
100% 316/316 [00:00<00:00, 1252.80it/s]
100% 316/316 [00:00<00:00, 363422.01it/s]


In [7]:
dataloader.dataset.class_name

['hot dog', 'dog']

## Training VGG16

In [8]:
input_size = (batch_size, 3, 224, 224)
num_classes = len(dataloader.dataset.class_name)
model = vgg16.VGG16(device, input_size=input_size, num_classes=num_classes)

Layer (type:depth-idx)                   Output Shape              Param #
Net                                      [32, 2]                   --
├─Sequential: 1-1                        [32, 64, 224, 224]        --
│    └─Conv2d: 2-1                       [32, 64, 224, 224]        1,792
│    └─BatchNorm2d: 2-2                  [32, 64, 224, 224]        128
│    └─ReLU: 2-3                         [32, 64, 224, 224]        --
├─Sequential: 1-2                        [32, 64, 112, 112]        --
│    └─Conv2d: 2-4                       [32, 64, 224, 224]        36,928
│    └─BatchNorm2d: 2-5                  [32, 64, 224, 224]        128
│    └─ReLU: 2-6                         [32, 64, 224, 224]        --
│    └─MaxPool2d: 2-7                    [32, 64, 112, 112]        --
├─Sequential: 1-3                        [32, 128, 112, 112]       --
│    └─Conv2d: 2-8                       [32, 128, 112, 112]       73,856
│    └─BatchNorm2d: 2-9                  [32, 128, 112, 112]       256
│

In [9]:
model_dir = 'coco2014_clf'
model.train(dataloader.dataset.trainloader, epochs=epochs, lr=learning_rate, wd=weight_decay, output_dir=model_dir)

[EPOCH #0] loss: 0.6153548272217021
[EPOCH #1, elapsed time: 3.962[sec]] loss: 7.26684949088239
[EPOCH #2, elapsed time: 7.730[sec]] loss: 0.2785912655732211
[EPOCH #3, elapsed time: 11.525[sec]] loss: 0.31644077489481254
[EPOCH #4, elapsed time: 15.359[sec]] loss: 0.22380714293788462
[EPOCH #5, elapsed time: 19.162[sec]] loss: 0.22321772772599668
[EPOCH #6, elapsed time: 22.993[sec]] loss: 0.5496498661882737
[EPOCH #7, elapsed time: 26.846[sec]] loss: 0.33894442323157015
[EPOCH #8, elapsed time: 30.698[sec]] loss: 0.24415693125304053
[EPOCH #9, elapsed time: 34.477[sec]] loss: 0.3100174824323724
[EPOCH #10, elapsed time: 38.290[sec]] loss: 0.2660251645480885
[EPOCH #11, elapsed time: 42.087[sec]] loss: 0.178917337944072
[EPOCH #12, elapsed time: 45.869[sec]] loss: 0.22078676236902967
[EPOCH #13, elapsed time: 49.661[sec]] loss: 0.15141517800443313
[EPOCH #14, elapsed time: 53.494[sec]] loss: 0.3577402900247013
[EPOCH #15, elapsed time: 57.282[sec]] loss: 0.22365090229055462
[EPOCH #16

## Test Model

In [10]:
train_result = model.predict(dataloader.dataset.trainloader)
train_predictions, train_labels = train_result

In [11]:
train_eval_result = model.evaluate(train_labels, train_predictions)
pprint.pprint(train_eval_result)

{'accuracy': 0.9229287090558767,
 'classification_report': {'0': {'f1-score': 0.0909090909090909,
                                 'precision': 1.0,
                                 'recall': 0.047619047619047616,
                                 'support': 42},
                           '1': {'f1-score': 0.9597585513078472,
                                 'precision': 0.9226305609284333,
                                 'recall': 1.0,
                                 'support': 477},
                           'accuracy': 0.9229287090558767,
                           'macro avg': {'f1-score': 0.525333821108469,
                                         'precision': 0.9613152804642167,
                                         'recall': 0.5238095238095238,
                                         'support': 519},
                           'weighted avg': {'f1-score': 0.8894470342813582,
                                            'precision': 0.9288916716047451,
                     

In [12]:
test_result = model.predict(dataloader.dataset.testloader)
test_predictions, test_labels = test_result

In [13]:
test_eval_result = model.evaluate(test_labels, test_predictions)
pprint.pprint(test_eval_result)

{'accuracy': 0.0379746835443038,
 'classification_report': {'0': {'f1-score': 0.0,
                                 'precision': 0.0,
                                 'recall': 0.0,
                                 'support': 85},
                           '1': {'f1-score': 0.07339449541284404,
                                 'precision': 0.03821656050955414,
                                 'recall': 0.9230769230769231,
                                 'support': 13},
                           '2': {'f1-score': 0.0,
                                 'precision': 0.0,
                                 'recall': 0.0,
                                 'support': 15},
                           '3': {'f1-score': 0.0,
                                 'precision': 0.0,
                                 'recall': 0.0,
                                 'support': 6},
                           '4': {'f1-score': 0.0,
                                 'precision': 0.0,
                            

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
